In [4]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.controls import *
from optic.manager import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *
from optic.visualization import *

class Suite2pROITrackingGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "SUITE2P_ROI_TRACKING"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager = initManagers(WidgetManager(), ConfigManager(), DataManager(), ControlManager())
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        self.initUI()

    """
    setup UI Function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)

        # FileLoadUI用のレイアウト
        self.layout_file_load = QVBoxLayout()
        self.setupFileLoadUI()
        self.layout_main.addLayout(self.layout_file_load, 1, 0, 1, 1)

        # メインUI用のレイアウト
        self.layout_main_ui = QGridLayout()
        self.layout_main.addLayout(self.layout_main_ui, 0, 0, 1, 1)

    def setupFileLoadUI(self):
        file_load_widget = QWidget()
        layout = QVBoxLayout(file_load_widget)
        # ファイル読み込み用のUIを追加
        layout.addLayout(self.makeLayoutSectionBottom())
        # bindFunc
        self.bindFuncFileLoadUI()

        self.layout_file_load.addWidget(file_load_widget)

    def loadFilePathsandInitialize(self):
        self.control_manager, self.data_manager = initManagers(self.control_manager, self.data_manager)
        success = self.loadData()
        if success:
            self.setupMainUI()
        else:
            return

    def setupMainUI(self):
        if self.setupUI_done:
            # メインUIのクリア
            clearLayout(self.layout_main_ui)
        
        # 新しいメインUIの設定
        self.setupMainUILayouts()
        self.setupControls()
        self.bindFuncAllWidget()

        self.setupUI_done = True

    def loadData(self):
        pass
        # success = loadFallMATWithGUI(
        #     q_window=self, 
        #     data_manager=self.data_manager, 
        #     key_app=self.app_key_pri, 
        #     path_fall=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_fall"].text()
        # )
        # if self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_reftif"].text() != "":
        #     self.data_manager.dict_im_bg_chan2[self.app_key_pri] = loadTIFImage(
        #         data_manager=self.data_manager, 
        #         key_dict_im_chan2=self.app_key_pri, 
        #         path_image=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_reftif"].text(), 
        #         preprocessing=True
        #         )
        # return success

    def setupMainUILayouts(self):
        pass
        # self.layout_main_ui.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        # self.layout_main_ui.addLayout(self.makeLayoutSectionRightUpper(), 0, 1, 1, 1)
        # self.layout_main_ui.addLayout(self.makeLayoutSectionCentral(), 1, 0, 1, 2)

    def setupControls(self):
        pass
        # self.control_manager.table_controls[self.app_key_pri] = TableControl(
        #                                                         key_app=self.app_key_pri,
        #                                                         q_table=self.widget_manager.dict_table[self.app_key_pri],
        #                                                         data_manager=self.data_manager,
        #                                                         widget_manager=self.widget_manager,
        #                                                         config_manager=self.config_manager,
        #                                                         control_manager=self.control_manager,
        #                                                         )
        
        # self.control_manager.table_controls[self.app_key_pri].setupWidgetROITable(self.app_key_pri)
        # self.control_manager.view_controls[self.app_key_pri] = ViewControl(
        #                                                             key_app=self.app_key_pri,
        #                                                             q_view=self.widget_manager.dict_view[self.app_key_pri], 
        #                                                             q_scene=self.widget_manager.dict_scene[self.app_key_pri], 
        #                                                             data_manager=self.data_manager, 
        #                                                             widget_manager=self.widget_manager,
        #                                                             config_manager=self.config_manager,
        #                                                             control_manager=self.control_manager,
        #                                                             )
        # self.control_manager.view_controls[self.app_key_pri].setViewSize()

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):
        for key_app in self.app_keys:
            list_key = [f"{self.app_key_pri}_path_fall", f"{self.app_key_pri}_path_reftif"]
            list_filetype = ["mat", "tiff"]
            for key, filetype in zip(list_key, list_filetype):
                bindFuncLoadFileWidget(q_widget=self, q_button=self.widget_manager.dict_button[key], q_lineedit=self.widget_manager.dict_lineedit[key], filetype=filetype)

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.loadFilePathsandInitialize())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        pass


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    gui = Suite2pROITrackingGUI()
    gui.show()
    sys.exit(app.exec_())

AttributeError: 'Suite2pROITrackingGUI' object has no attribute 'makeLayoutSectionBottom'

In [3]:
gui.app_keys

['pri', 'sec']